In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report

# 0. Set seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# 0.1 Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 1. Data Generators
img_size = (299, 299)  # InceptionV3 requires 299x299 input
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    'hollywood_train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_gen = val_datagen.flow_from_directory(
    'hollywood_validation',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Print class info
print(f"Number of training classes: {train_gen.num_classes}")
print(f"Training class indices: {train_gen.class_indices}")
print(f"Number of validation classes: {val_gen.num_classes}")
print(f"Validation class indices: {val_gen.class_indices}")

# 2. Model Building (Inception + LeakyReLU)
num_classes = train_gen.num_classes
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 3. Callbacks
''''callbacks = [
    EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model_inception.h5', save_best_only=True, monitor='val_accuracy')
]'''

# 4. Training
history = model.fit(
    train_gen,
    epochs=99,
    validation_data=val_gen,
    callbacks=callbacks
)

# 5. Evaluation
loss, acc = model.evaluate(val_gen)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {acc:.4f}")

# 5.1 Classification Report
y_true = val_gen.classes
val_gen.reset()
Y_pred = model.predict(val_gen)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))

# 6. Save final model
model.save('final_model_inception.h5')


Found 1440 images belonging to 17 classes.
Found 360 images belonging to 17 classes.
Number of training classes: 17
Training class indices: {'Angelina Jolie': 0, 'Brad Pitt': 1, 'Denzel Washington': 2, 'Hugh Jackman': 3, 'Jennifer Lawrence': 4, 'Johnny Depp': 5, 'Kate Winslet': 6, 'Leonardo DiCaprio': 7, 'Megan Fox': 8, 'Natalie Portman': 9, 'Nicole Kidman': 10, 'Robert Downey Jr': 11, 'Sandra Bullock': 12, 'Scarlett Johansson': 13, 'Tom Cruise': 14, 'Tom Hanks': 15, 'Will Smith': 16}
Number of validation classes: 17
Validation class indices: {'Angelina Jolie': 0, 'Brad Pitt': 1, 'Denzel Washington': 2, 'Hugh Jackman': 3, 'Jennifer Lawrence': 4, 'Johnny Depp': 5, 'Kate Winslet': 6, 'Leonardo DiCaprio': 7, 'Megan Fox': 8, 'Natalie Portman': 9, 'Nicole Kidman': 10, 'Robert Downey Jr': 11, 'Sandra Bullock': 12, 'Scarlett Johansson': 13, 'Tom Cruise': 14, 'Tom Hanks': 15, 'Will Smith': 16}


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_282 (Conv2D) │ (None, 149, 149,  │        864 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 149, 149,  │         96 │ conv2d_282[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_282      │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_283 (Conv2D) │ (None, 147, 147,  │      9,216 │ activation_282[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_283[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_283      │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_284 (Conv2D) │ (None, 147, 147,  │     18,432 │ activation_283[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_284[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_284      │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 73, 73,    │          0 │ activation_284[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_285 (Conv2D) │ (None, 73, 73,    │      5,120 │ max_pooling2d_12… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_285[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_285      │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_286 (Conv2D) │ (None, 71, 71,    │    138,240 │ activation_285[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_286[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_286      │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,331,697 (85.19 MB)

 Trainable params: 528,913 (2.02 MB)

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6095 - loss: 1.1194 - val_accuracy: 0.6083 - val_loss: 1.2836
Epoch 52/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.6470 - loss: 1.0767 - val_accuracy: 0.5917 - val_loss: 1.2804
Epoch 53/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.6549 - loss: 1.0643 - val_accuracy: 0.5944 - val_loss: 1.2886
Epoch 54/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.6485 - loss: 1.0732 - val_accuracy: 0.5944 - val_loss: 1.2658
Epoch 55/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6783 - loss: 1.0589 - val_accuracy: 0.5944 - val_loss: 1.2690
Epoch 56/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6517 - loss: 1.0746 - val_accuracy: 0.6056 - val_loss: 1.2565
Epoch 57/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6467 - loss: 1.0849

45/45 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6466 - loss: 1.0845 - val_accuracy: 0.6139 - val_loss: 1.2458
Epoch 58/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 983ms/step - accuracy: 0.6566 - loss: 1.0303

45/45 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.6565 - loss: 1.0305 - val_accuracy: 0.6167 - val_loss: 1.2421
Epoch 59/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 997ms/step - accuracy: 0.6480 - loss: 1.0114

45/45 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.6480 - loss: 1.0117 - val_accuracy: 0.6194 - val_loss: 1.2346
Epoch 60/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.6630 - loss: 0.9961 - val_accuracy: 0.6194 - val_loss: 1.2359
Epoch 61/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.6679 - loss: 0.9908 - val_accuracy: 0.6167 - val_loss: 1.2426
Epoch 62/99
45/45 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.6663 - loss: 0.9800 - val_accuracy: 0.6194 - val_loss: 1.2497
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 876ms/step - accuracy: 0.5971 - loss: 1.2964
Validation Loss: 1.2346, Validation Accuracy: 0.6194
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 998ms/step


                    precision    recall  f1-score   support

    Angelina Jolie       0.05      0.05      0.05        20
         Brad Pitt       0.00      0.00      0.00        20
 Denzel Washington       0.11      0.15      0.13        20
      Hugh Jackman       0.09      0.05      0.06        20
 Jennifer Lawrence       0.11      0.10      0.10        20
       Johnny Depp       0.06      0.05      0.05        20
      Kate Winslet       0.06      0.05      0.05        20
 Leonardo DiCaprio       0.05      0.05      0.05        20
         Megan Fox       0.00      0.00      0.00        20
   Natalie Portman       0.06      0.05      0.05        20
     Nicole Kidman       0.10      0.05      0.07        20
  Robert Downey Jr       0.00      0.00      0.00        20
    Sandra Bullock       0.19      0.15      0.17        20
Scarlett Johansson       0.08      0.12      0.09        40
        Tom Cruise       0.09      0.10      0.10        20
         Tom Hanks       0.00      0.00

using augmentation

In [5]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report

# 0. Set seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# 0.1 Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 1. Data Generators with Strong Augmentation for Training
img_size = (299, 299)  # InceptionV3 preferred size
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    'hollywood_train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    'hollywood_validation',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Print class info
print(f"Number of training classes: {train_gen.num_classes}")
print(f"Training class indices: {train_gen.class_indices}")
print(f"Number of validation classes: {val_gen.num_classes}")
print(f"Validation class indices: {val_gen.class_indices}")

# 2. Model Building
num_classes = train_gen.num_classes
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 3. Callbacks
'''callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model_inception_aug.h5', save_best_only=True, monitor='val_accuracy')
]
'''
# 4. Training
history = model.fit(
    train_gen,
    epochs=90,
    validation_data=val_gen,
    callbacks=callbacks
)

# 5. Evaluation
loss, acc = model.evaluate(val_gen)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {acc:.4f}")

# 5.1 Classification Report
y_true = val_gen.classes
val_gen.reset()
Y_pred = model.predict(val_gen)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))

# 6. Save final model
model.save('final_model_inception_augmented.h5')


Found 1440 images belonging to 17 classes.
Found 360 images belonging to 17 classes.
Number of training classes: 17
Training class indices: {'Angelina Jolie': 0, 'Brad Pitt': 1, 'Denzel Washington': 2, 'Hugh Jackman': 3, 'Jennifer Lawrence': 4, 'Johnny Depp': 5, 'Kate Winslet': 6, 'Leonardo DiCaprio': 7, 'Megan Fox': 8, 'Natalie Portman': 9, 'Nicole Kidman': 10, 'Robert Downey Jr': 11, 'Sandra Bullock': 12, 'Scarlett Johansson': 13, 'Tom Cruise': 14, 'Tom Hanks': 15, 'Will Smith': 16}
Number of validation classes: 17
Validation class indices: {'Angelina Jolie': 0, 'Brad Pitt': 1, 'Denzel Washington': 2, 'Hugh Jackman': 3, 'Jennifer Lawrence': 4, 'Johnny Depp': 5, 'Kate Winslet': 6, 'Leonardo DiCaprio': 7, 'Megan Fox': 8, 'Natalie Portman': 9, 'Nicole Kidman': 10, 'Robert Downey Jr': 11, 'Sandra Bullock': 12, 'Scarlett Johansson': 13, 'Tom Cruise': 14, 'Tom Hanks': 15, 'Will Smith': 16}


c:\Users\vinay\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,331,697 (85.19 MB)

 Trainable params: 528,913 (2.02 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

NameError: name 'callbacks' is not defined

In [4]:
import gradio as gr
import numpy as np
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Use the already loaded model and val_gen from previous cells

# Get class labels from val_gen
class_labels = list(val_gen.class_indices.keys())

# Prediction function
def predict_image(img):
    img = img.resize((299, 299))  # Resize for InceptionV3
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    confidences = {class_labels[i]: float(preds[0][i]) for i in range(len(class_labels))}
    return confidences

# Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="Celebrity Face Classifier",
    description="Upload an image of a face and the model will predict which celebrity it is."
)

interface.launch()


NameError: name 'val_gen' is not defined